In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import numpy as np
import tensorflow as tf
import nltk
nltk.download('punkt')
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import Embedding, LSTM, Dense
from tensorflow.keras.models import Sequential
from tensorflow.keras.utils import to_categorical
import re
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [3]:
!pip install nltk

In [4]:
file = open("/content/drive/MyDrive/text_shona.csv","r")
text_shona = file.read()

In [5]:
text_shona

'"Mudunhu rekwaChitororo yaiva burushashike vanhu vachifarira muroora mutsva anga awuya mumusha wekwaNhoto. Ivo vaNhoto vainzwa manyuku nyuku nekuti mwanakomana wavo mukuru Wayne anga avaunzira ndangaruvazhe. Wayne anga avakushanda mudhorobha riripedyo nedunhu ravo. Kubva paakapedza zvidzidzo zvake zveOlevel akabva angoenda kudhorobha kunoshava. Akabva awana basa kwaPower Sales. Ikoko kudhorobha ndiko kwaakasangana netsvarakadenga yake yainzi Rosemary Chuma. Vakafambidzana mwedzi yakati kuti kusvika vaviri ava vapanana pamuviri. Vakabva vatoronga zvekutizisana. MaChuma akabva atora vana tete vake vaviri vakava ndivo vakamuperedza. Pavakasvika mudunhu raChitororo vakambogara pamba pabamunini vaWayne mazuva matatu muroora asati aperekwa mumusha memurume wake. Zuva rekuperekwa kwake rakazosvika. Munhu wese mudunhu iri anga akarimirira zuva iri. Dunhu rese rakaturunuka kuzoona svusvurandadya yanga yaunzwa naWayne mumusha wekwaNhoto. Musikana anga akaumbwa hake iyeye. Ndivo vanonzi Mwari va

In [6]:
# Preprocess the text
def clean_text(text_shona):
    # Convert text to lowercase
    text_shona = text_shona.lower()

    # Remove punctuation and special characters
    text_shona = re.sub(r'[^\w\s]', '', text_shona)

    # Tokenize the text
    words = word_tokenize(text_shona)

    # Remove stopwords
    nltk.download('stopwords')
    stop_words = set(stopwords.words('english'))
    words = [word for word in words if word not in stop_words]

    # Join the words back into a cleaned text
    cleaned_text = ' '.join(words)

    return cleaned_text

cleaned_text = clean_text(text_shona)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [7]:
print(cleaned_text)

mudunhu rekwachitororo yaiva burushashike vanhu vachifarira muroora mutsva anga awuya mumusha wekwanhoto ivo vanhoto vainzwa manyuku nyuku nekuti mwanakomana wavo mukuru wayne anga avaunzira ndangaruvazhe wayne anga avakushanda mudhorobha riripedyo nedunhu ravo kubva paakapedza zvidzidzo zvake zveolevel akabva angoenda kudhorobha kunoshava akabva awana basa kwapower sales ikoko kudhorobha ndiko kwaakasangana netsvarakadenga yake yainzi rosemary chuma vakafambidzana mwedzi yakati kuti kusvika vaviri ava vapanana pamuviri vakabva vatoronga zvekutizisana machuma akabva atora vana tete vake vaviri vakava ndivo vakamuperedza pavakasvika mudunhu rachitororo vakambogara pamba pabamunini vawayne mazuva matatu muroora asati aperekwa mumusha memurume wake zuva rekuperekwa kwake rakazosvika munhu wese mudunhu iri anga akarimirira zuva iri dunhu rese rakaturunuka kuzoona svusvurandadya yanga yaunzwa nawayne mumusha wekwanhoto musikana anga akaumbwa hake iyeye ndivo vanonzi mwari vakatora nguva yav

In [8]:
cleaned_text

'mudunhu rekwachitororo yaiva burushashike vanhu vachifarira muroora mutsva anga awuya mumusha wekwanhoto ivo vanhoto vainzwa manyuku nyuku nekuti mwanakomana wavo mukuru wayne anga avaunzira ndangaruvazhe wayne anga avakushanda mudhorobha riripedyo nedunhu ravo kubva paakapedza zvidzidzo zvake zveolevel akabva angoenda kudhorobha kunoshava akabva awana basa kwapower sales ikoko kudhorobha ndiko kwaakasangana netsvarakadenga yake yainzi rosemary chuma vakafambidzana mwedzi yakati kuti kusvika vaviri ava vapanana pamuviri vakabva vatoronga zvekutizisana machuma akabva atora vana tete vake vaviri vakava ndivo vakamuperedza pavakasvika mudunhu rachitororo vakambogara pamba pabamunini vawayne mazuva matatu muroora asati aperekwa mumusha memurume wake zuva rekuperekwa kwake rakazosvika munhu wese mudunhu iri anga akarimirira zuva iri dunhu rese rakaturunuka kuzoona svusvurandadya yanga yaunzwa nawayne mumusha wekwanhoto musikana anga akaumbwa hake iyeye ndivo vanonzi mwari vakatora nguva ya

Vocabulary

In [11]:
#using a keras tokenizer to build an optimal vocabulary
from tensorflow.keras.preprocessing.text import Tokenizer
tokenizer = Tokenizer()
tokenizer.fit_on_texts(text_shona)
max_vocab_size= 10000  # the vocabulary size
word_index = tokenizer.word_index
vocabulary_size = min(max_vocab_size, len(word_index))
print(vocabulary_size)

24


Word Embeddings

In [12]:
!pip install gensim

In [13]:
import gensim
from gensim.models import Word2Vec
# Train word2vec model with gensim
sentences = [sentence.split() for sentence in cleaned_text.split('.')]
model_gensim = Word2Vec(sentences, vector_size=100, window=5, min_count=1, workers=4)
model_gensim.save("word2vec.model")

 RNN Models with its own embedding layer

In [14]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Bidirectional, LSTM, Dense, Dropout
#the maximum sequence length and embedding dimension
max_sequence_length = 100
embedding_dim = 100
model1 = Sequential()
# Embedding layer
model1.add(Embedding(input_dim=vocabulary_size, output_dim=embedding_dim, input_length=max_sequence_length))
model1.add(Bidirectional(LSTM(units=150, return_sequences=True)))
model1.add(Dropout(0.2))
model1.add(Bidirectional(LSTM(units=100)))
model1.add(Dense(units=64, activation='relu'))
model1.add(Dense(max_vocab_size, activation='softmax'))
model1.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model1.summary()


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 100, 100)          2400      
                                                                 
 bidirectional (Bidirection  (None, 100, 300)          301200    
 al)                                                             
                                                                 
 dropout (Dropout)           (None, 100, 300)          0         
                                                                 
 bidirectional_1 (Bidirecti  (None, 200)               320800    
 onal)                                                           
                                                                 
 dense (Dense)               (None, 64)                12864     
                                                                 
 dense_1 (Dense)             (None, 10000)             6

With Pre-Trained Embeddings

In [15]:
#saving model 1
model1.save('model1.h5')

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [17]:
# Load the word2vec model
import numpy as np
model_gensim = Word2Vec.load("word2vec.model")
embedding_matrix = np.zeros((max_vocab_size, 100))
for word, i in tokenizer.word_index.items():
    try:
        embedding_vector = model_gensim.wv[word]
        if embedding_vector is not None:
            embedding_matrix[i] = embedding_vector
    except KeyError:
        # Word not present in gensim model, a zero embedding will be used for this word
        pass


model2 = Sequential()
model2.add(Embedding(max_vocab_size, 100, weights=[embedding_matrix], input_length=5, trainable=False))
model2.add(Bidirectional(LSTM(150, return_sequences=True)))
model2.add(Dropout(0.2))
model2.add(LSTM(100))
model2.add(Dense(max_vocab_size, activation='softmax'))

model2.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])


In [18]:
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Bidirectional, LSTM, Dense, Dropout

max_sequence_length = 100
embedding_dim = 100

model2 = Sequential()

# Embedding layer (defined in (3))
model2.add(Embedding(input_dim=max_vocab_size, output_dim=embedding_dim, input_length=max_sequence_length, weights=[embedding_matrix], trainable=False))
model2.add(Bidirectional(LSTM(units=150, return_sequences=True)))
model2.add(Dropout(0.2))
model2.add(Bidirectional(LSTM(units=100)))
model2.add(Dropout(0.2))
model2.add(Dense(units=64, activation='relu'))
model2.add(Dense(units=max_vocab_size, activation='softmax'))
model2.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model2.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_2 (Embedding)     (None, 100, 100)          1000000   
                                                                 
 bidirectional_3 (Bidirecti  (None, 100, 300)          301200    
 onal)                                                           
                                                                 
 dropout_2 (Dropout)         (None, 100, 300)          0         
                                                                 
 bidirectional_4 (Bidirecti  (None, 200)               320800    
 onal)                                                           
                                                                 
 dropout_3 (Dropout)         (None, 200)               0         
                                                                 
 dense_3 (Dense)             (None, 64)               

In [19]:
#saving model 2
model2.save('model2.h5')

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [20]:
from tensorflow.keras.models import load_model
import numpy as np
import pickle

#Load the model and tokenizer
model1=load_model('model1.h5')
model2=load_model('model2.h5')

Training & Testing

In [21]:
import keras
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split

input_sequences = []
for line in cleaned_text.split('.'):
    token_list = tokenizer.texts_to_sequences([line])[0]
    for i in range(1, len(token_list)):
        n_gram_sequence = token_list[:i+1]
        input_sequences.append(n_gram_sequence)

input_sequences = np.array(pad_sequences(input_sequences, maxlen=6, padding='pre'))
X, y = input_sequences[:, :-1], input_sequences[:, -1]
y = keras.utils.to_categorical(y, num_classes=max_vocab_size)

# Split the dataset into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.5, random_state=42)

# Training Model 1
history1 = model1.fit(X_train, y_train, validation_data=(X_val, y_val), epochs=100, verbose=1)

# Training Model 2
history2 = model2.fit(X_train, y_train, validation_data=(X_val, y_val), epochs=100, verbose=1)

ValueError: ignored

Model Evaluation

In [ ]:
val_loss_model1 = history1.history['val_loss'][-1]
val_loss_model2 = history2.history['val_loss'][-1]

print(f"Validation Loss for Model 1: {val_loss_model1}")
print(f"Validation Loss for Model 2: {val_loss_model2}")

In [ ]:
if val_loss_model1 < val_loss_model2:
    best_model = model1
    best_model_name = "best_model1.h5"
else:
    best_model = model2
    best_model_name = "best_model2.h5"

best_model.save(best_model_name)
print(f"Saved the best model as {best_model_name}")

In [ ]:
params_model1 = model1.count_params()
params_model2 = model2.count_params()

print(f"Model 1 has {params_model1} parameters.")
print(f"Model 2 has {params_model2} parameters.")

In [ ]:
from keras.models import load_model
from tensorflow.keras.preprocessing.sequence import pad_sequences

# Load the previously saved model
model = load_model('best_model1.h5')

def predict_next_words(model, tokenizer, text, num_words=1):
    """
    Predict the next set of words using the trained model.

    Args:
    - model (keras.Model): The trained model.
    - tokenizer (Tokenizer): The tokenizer object used for preprocessing.
    - text (str): The input text.
    - num_words (int): The number of words to predict.

    Returns:
    - str: The predicted words.
    """
    for _ in range(num_words):
        # Tokenize and pad the text
        sequence = tokenizer.texts_to_sequences([text])[0]
        sequence = pad_sequences([sequence], maxlen=5, padding='pre')

        # Predict the next word
        predicted_probs = model.predict(sequence, verbose=0)
        predicted = np.argmax(predicted_probs, axis=-1)

        # Convert the predicted word index to a word
        output_word = ""
        for word, index in tokenizer.word_index.items():
            if index == predicted:
                output_word = word
                break

        # Append the predicted word to the text
        text += " " + output_word

    return ' '.join(text.split(' ')[-num_words:])


# Prompt the user for input
user_input = input("Please type five words in Shona: ")

# Predict the next words
predicted_words = predict_next_words(model, tokenizer, user_input, num_words=3)
print(f"The next words might be: {predicted_words}")

Deploying